In [ ]:
from bluesky import RunEngine
from bluesky.plans import scan, grid_scan
from bluesky.plan_stubs import mv, open_run, close_run, \
    trigger_and_read
from bluesky.callbacks import LiveTable
from bluesky.preprocessors import relative_set_decorator, \
    reset_positions_decorator, run_decorator, stage_decorator, \
    finalize_decorator
from ophyd import Device, Component
from ophyd.sim import motor1, motor2, det1, det4, FakeEpicsSignal

RE = RunEngine()

## Decorators

In [ ]:
@bluesky_decorator
def custom_plan(*args):
    # Customized plan
    pass

custom_plan(*args)

In [ ]:
bluesky_decorator(custom_plan)(*args)

def bluesky_decorator(*args):
    yield from decorator_before()
    
    yield from custom_plan(*args)
    
    yield from decorator_after()

## relative_set_decorator

In [ ]:
help(relative_set_decorator)

In [ ]:
@relative_set_decorator([motor1])
def relative_movement():
    yield from open_run()
    
    yield from mv(motor1, -2)
    print(motor1.readback.get())
    
    yield from mv(motor1, 3)
    print(motor1.readback.get())
    
    yield from mv(motor1, 1.5)
    print(motor1.readback.get())
    
    yield from close_run()

RE(mv(motor1, 5))
RE(relative_movement(), LiveTable(['motor1']))

In [ ]:
@relative_set_decorator()
def relative_grid_scan():
    yield from grid_scan(
        [det4], motor1, -5, 1, 5, motor2, -3, 2, 5)

RE(mv(
    motor1, 1.5,
    motor2, 2
))
RE(relative_grid_scan(), LiveTable(['det4', 'motor1', 'motor2']))

## reset_positions_decorator

In [ ]:
help(reset_positions_decorator)

In [ ]:
@reset_positions_decorator([motor1])
def reset_m1_position_scan():
    yield from scan([det1], motor1, -10, 10, 10)
    yield from mv(motor2, 5)

In [ ]:
RE(mv(motor1, 0, motor2, 0))
RE(reset_m1_position_scan())
print(f"Motor 1: {motor1.readback.get()}")
print(f"Motor 2: {motor2.readback.get()}")

In [ ]:
@reset_positions_decorator([motor1, motor2])
def reset_m1_and_m2_position_scan():
    yield from scan([det1], motor1, -10, 10, 10)
    yield from mv(motor2, 5)

In [ ]:
RE(mv(motor1, 0, motor2, 0))
RE(reset_m1_and_m2_position_scan())
print(f"Motor 1: {motor1.readback.get()}")
print(f"Motor 2: {motor2.readback.get()}")

## finalize_decorator

In [ ]:
help(finalize_decorator)

In [ ]:
class NoLiquidAgentError(Exception):
    pass

class LiquidAgent(Device):
     active = Component(FakeEpicsSignal, "Active", kind="omitted")

def print_agent_states():
    print(f"Agent 1: {agent1.active.get()}")
    print(f"Agent 2: {agent2.active.get()}")
    print()
        
def safe_plan_stop():
    yield from mv(agent1.active, False)
    yield from mv(agent2.active, False)
    print("The plan has ended safely")
    
agent1 = LiquidAgent("SIM:", name="agent1")
agent2 = LiquidAgent("SIM:", name="agent2")

In [ ]:
@finalize_decorator(safe_plan_stop)
def plan_with_exception():
    yield from mv(agent1.active, True)
    print("Agent 1 activated")
    print_agent_states()
    
    raise NoLiquidAgentError()
    
    yield from mv(agent2.active, True)
    print("Agent 2 activated")
    print_agent_states()
    
try:
    RE(plan_with_exception())
except:
    print(f"Run Engine State: {RE.state}")
    print_agent_states()

## run_decorator

In [ ]:
help(run_decorator)

In [ ]:
@run_decorator()
def plan_run():
    yield from mv(motor1, -10)
    yield from trigger_and_read([det1, motor1])
    
    yield from mv(motor1, 0)
    yield from trigger_and_read([det1, motor1])
    
    yield from mv(motor1, 10)
    yield from trigger_and_read([det1, motor1])
    
RE(
    plan_run(), LiveTable(['det1', 'motor1']))